# From OMOP to IDEA4RC Radiotherapy NEED TO FINISH

we have the problem with the neck and neck surgeries that still needs further calrification



In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta
from uuid import uuid4

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### From OMOP to Radiotherapy

We will be doing all the steps at once so it is more simple

WHAT TO DO WITH THE CUSTOM CODES? We need tons of concept ids

In [ ]:
selectProcedures="""
SELECT 
    procedure_occurrence_id, procedure_concept_id, procedure_date, provider_id, procedure_end_date 
    FROM procedure_occurrence
    WHERE procedure_concept_id IN (4059384, 4058775);
"""

selectEpisodeData="""
    SELECT episode_concept_id, parent_episode_id FROM episode WHERE episode_id IN (
    SELECT episode_id FROM episode_event WHERE event_id=%s AND episode_event_field_concept_id=1147082);
"""

selectDataParentEpisode="""
    SELECT episode_start_date FROM episode WHERE episode_id=%s;
"""

selectEpisodeEventIDEA="""
    SELECT id FROM episode_event WHERE cancer_episode IN (
    SELECT id FROM cancer_episode WHERE patient_id=%s AND date_of_diagnosis=%s);
"""

getCareSiteName="""
    SELECT care_site_name FROM care_site WHERE care_site_id IN(SELECT care_site_id FROM provider WHERE provider_id=%s);
"""

measurementsConceptIds={
    "primaryOnly":????,
    "neckOnly":????,
    "primaryAndIpsilateralNeck":????,
    "primaryAndBilateralNeck":????,
    "distantMetastasis":????,
    "lung":????,
    "mediastinum":????,
    "bone":????,
    "softTissue":????,
    "liver":????,
    "treatmentCompletedAsPlanned":(4058777,438028,4162706,4105297,4306655),
    "beamQuality":(607996,4165039,607995,4024005,4012485,4161415,4058601,4077953),
    "otherSpecify":????,
    "treatmentTechnique":(603132,37156151,40480519,603135,3662120,4215577,37156211,4077990),
    "totalDoseGy":????????, #integer
    "numberOfFractions":4037631,  #integer
    "adaptiveRt":????,
    "igrt":????
}

selectObservation="""
    SELECT observation_concept_id FROM observation WHERE observation_concept_id IN (4179711,4162591) AND observation_event_id=%s AND obs_event_field_concept_id=1147082;
"""

curOMOP.execute(selectProcedures)
procedures=curOMOP.fetchall()

insertRadiotherapy="""
    INSERT INTO radiotherapy (episode_event, hospital, setting, start_date, end_date, clinical_treatment_response) VALUES (%s, %s, %s, %s, %s, %s) RETURNING id;
"""
booleans=[
    "adaptiveRt",
    "igrt",
    "primaryOnly",
    "neckOnly",
    "primaryAndIpsilateralNeck",
    "primaryAndBilateralNeck",
    "distantMetastasis",
    "lung",
    "mediastinum",
    "bone",
    "softTissue",
    "liver"
]
for procedure in procedures:
    curOMOP.execute(selectEpisodeData, (procedure[0]))
    episodeData=curOMOP.fetchone()
    curOMOP.execute(selectDataParentEpisode, (episodeData[1]))
    episodeStartDate=curOMOP.fetchone()
    curIDEA.execute(selectEpisodeEventIDEA, (episodeData[1], episodeStartDate[0]))
    episodeEventId=curIDEA.fetchone()
    curOMOP.execute(getCareSiteName, (procedure[3]))
    careSiteName=curOMOP.fetchone()
    curIDEA.execute(insertRadiotherapy, (episodeEventId[0], careSiteName[0], procedure[1], procedure[2], procedure[4], episodeData[0]))
    radiotherapyId=curIDEA.fetchone()
    conn2.commit()
    for column in measurementsConceptIds.keys():
        values=measurementsConceptIds[column]
        queryGetMeasurement=f"""
            SELECT measuement_concept_id, value_as_number, value_as_concept_id FROM measurement WHERE measurement_concept_id IN {values} AND measurement_event_id={procedure[0]} AND meas_event_field_concept_id=1147082;
        """
        curOMOP.execute(queryGetMeasurement)
        measurement=curOMOP.fetchone()
        insert=f"""
            UPDATE radiotherapy SET {column}=%s WHERE id=%s;
        """
        valToInsert=None
        if column=='totalDoseGy' or column=='numberOfFractions':
            valToInsert=measurement[1]
        elif column in booleans:
            valToInsert=measurement[2]
        else:
            valToInsert=measurement[0]
        curIDEA.execute(insert, (valToInsert, radiotherapyId[0]))
        conn2.commit()
    
    #Inserting observations
    curOMOP.execute(selectObservation, (procedure[0]))
    observation=curOMOP.fetchone()
    insert="""
        UPDATE radiotherapy SET intent=%s WHERE id=%s;
    """
    curIDEA.execute(insert, (observation[0], radiotherapyId[0]))
    conn2.commit()